In [ ]:
from google.colab import drive
import pandas as pd
import os

# 드라이브 연결
drive.mount('/content/drive')

# 기본 경로
DATA_DIR = "/content/drive/MyDrive/aiary/data"

# 파일 리스트 확인
print(os.listdir(DATA_DIR))


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
['감성대화말뭉치(최종데이터)_Validation.xlsx', 'diary_corpus.csv', 'diary_corpus_parenting_files_only.csv', 'diary_corpus_parenting_final.csv', 'diary_corpus_parenting_final_splitted.csv', 'diary_corpus_parenting_file_level.csv', 'diary_corpus_parenting_sentence_level.csv', '감성대화말뭉치(최종데이터)_Training.xlsx', '감성대화말뭉치_filtered', 'parenting_diary_gpt', 'parenting_diary_gpt_fail']


In [ ]:
train_path = os.path.join(DATA_DIR, "감성대화말뭉치(최종데이터)_Training.xlsx")
val_path   = os.path.join(DATA_DIR, "감성대화말뭉치(최종데이터)_Validation.xlsx")

train_df = pd.read_excel(train_path)
val_df = pd.read_excel(val_path)

print("Train:", train_df.shape)
print("Val:", val_df.shape)
train_df.head()


Train: (51630, 13)
Val: (6641, 13)


,Unnamed: 0,연령,성별,상황키워드,신체질환,감정_대분류,감정_소분류,사람문장1,시스템문장1,사람문장2,시스템문장2,사람문장3,시스템문장3
0,1,청년,여성,"진로,취업,직장",해당없음,분노,노여워하는,일은 왜 해도 해도 끝이 없을까? 화가 난다.,많이 힘드시겠어요. 주위에 의논할 상대가 있나요?,그냥 내가 해결하는 게 나아. 남들한테 부담 주고 싶지도 않고.,혼자 해결하기로 했군요. 혼자서 해결하기 힘들면 주위에 의논할 사람을 찾아보세요.,NaN,NaN
1,2,청년,여성,"진로,취업,직장",해당없음,분노,노여워하는,이번 달에 또 급여가 깎였어! 물가는 오르는데 월급만 자꾸 깎이니까 너무 화가 나.,급여가 줄어 속상하시겠어요. 월급이 줄어든 것을 어떻게 보완하실 건가요?,최대한 지출을 억제해야겠어. 월급이 줄어들었으니 고정지출을 줄일 수밖에 없을 것 같아.,월급이 줄어든 만큼 소비를 줄일 계획이군요.,NaN,NaN
2,3,청년,여성,"진로,취업,직장",해당없음,분노,노여워하는,회사에 신입이 들어왔는데 말투가 거슬려. 그런 애를 매일 봐야 한다고 생각하니까 스...,회사 동료 때문에 스트레스를 많이 받는 것 같아요. 문제 해결을 위해 어떤 노력을 ...,잘 안 맞는 사람이랑 억지로 잘 지내는 것보단 조금은 거리를 두고 예의를 갖춰서 대...,스트레스받지 않기 위해선 인간관계에 있어 약간의 거리를 두는 게 좋겠군요.,NaN,NaN
3,4,청년,여성,"진로,취업,직장",해당없음,분노,노여워하는,직장에서 막내라는 이유로 나에게만 온갖 심부름을 시켜. 일도 많은 데 정말 분하고 ...,관련 없는 심부름을 모두 하게 되어서 노여우시군요. 어떤 것이 상황을 나아질 수 있...,직장 사람들과 솔직하게 이야기해보고 싶어. 일하는 데에 방해된다고.,직장 사람들과 이야기를 해 보겠다고 결심하셨군요.,NaN,NaN
4,5,청년,여성,"진로,취업,직장",해당없음,분노,노여워하는,얼마 전 입사한 신입사원이 나를 무시하는 것 같아서 너무 화가 나.,무시하는 것 같은 태도에 화가 나셨군요. 상대방의 어떤 행동이 그런 감정을 유발하는...,상사인 나에게 먼저 인사하지 않아서 매일 내가 먼저 인사한다고!,항상 먼저 인사하게 되어 화가 나셨군요. 어떻게 하면 신입사원에게 화났음을 표현할 ...,NaN,NaN


In [ ]:
import pandas as pd
import re

# 1) 키워드 정의
parent_keywords = [
    "아기","애기","아가","신생아","영아","유아","아이","우리 아이","우리 애",
    "아들","딸",
    "기저귀","수유","모유","분유","이유식","배변훈련",
    "걸음마","옹알이","등원","하원","어린이집","유치원",
    "육아","독박육아","육퇴",
    "떼쓰기","보채","안아주","달래"
]

exclude_keywords = [
    "강아지","고양이","반려견","산책",
    "남자친구","여자친구","연애",
    "직장","회사","업무","시험","학교","학원"
]

parent_pattern = re.compile("|".join(map(re.escape, parent_keywords)))
exclude_pattern = re.compile("|".join(map(re.escape, exclude_keywords)))

# 2) 대화 전체 병합
def merge_dialogue(row):
    fields = ["사람문장1","시스템문장1","사람문장2","시스템문장2","사람문장3","시스템문장3"]
    return " ".join(str(row[f]) for f in fields if pd.notna(row[f]))

def is_parenting_dialogue(text):
    # 제외 키워드 먼저 체크
    if exclude_pattern.search(text):
        return False
    # 포함 키워드 하나라도 포함되면 육아 대화
    return bool(parent_pattern.search(text))

# 3) 필터링 적용
train_df["dialogue"] = train_df.apply(merge_dialogue, axis=1)
val_df["dialogue"] = val_df.apply(merge_dialogue, axis=1)

train_parent = train_df[train_df["dialogue"].apply(is_parenting_dialogue)]
val_parent = val_df[val_df["dialogue"].apply(is_parenting_dialogue)]

print("훈련 육아 대화 수:", len(train_parent))
print("검증 육아 대화 수:", len(val_parent))


훈련 육아 대화 수: 6017
검증 육아 대화 수: 693


In [ ]:
sample = train_parent[["상황키워드","감정_대분류","감정_소분류","dialogue"]].head(10)
sample


,상황키워드,감정_대분류,감정_소분류,dialogue
93,"진로,취업,직장",슬픔,우울한,보육교사라는 직업이 점점 힘들어지는 것 같아. 점점 우울하네. 보육교사라서 힘든 일...
97,"연애,결혼,출산",당황,한심한,결혼하고 아이가 생기면 좋을 줄 알았는데 그런 건 아무것도 없었어. 상상하던 일들과...
100,"연애,결혼,출산",기쁨,기쁨,결혼한 지 삼 년 만에 아이를 낳았어. 진짜 기뻐. 간절히 바라던 아이를 출산하셔서...
192,"연애,결혼,출산",당황,혼란스러운,드디어 내가 아빠가 된다니 상상해보면 생각처럼 쉽지가 않을 것 같아. 드디어 아빠가...
198,대인관계,상처,질투하는,부잣집 아들인 친구가 부러우면서도 질투나. 부잣집 아들인 친구가 부럽기도 하시고 질...
206,"연애,결혼,출산",당황,외로운,삼 년 동안 기러기 아빠로 지내다 보니 외롭고 지쳐. 아내와 아들이 너무 보고 싶어...
214,대인관계,분노,성가신,성가신 친구가 하나 있는데 너무 귀찮아. 그렇군요. 그 친구는 어떻게 성가시게 하나...
221,"연애,결혼,출산",당황,부끄러운,아내가 아기를 낳았는데 너무 고통스러워했어. 아내의 모습을 보니 아무것도 할 수 없...
222,"연애,결혼,출산",상처,상처,아내가 나보고 왜 본인에게만 육아를 맡기냐며 화를 내더라. 너무 속상했어. 많이 노...
224,"연애,결혼,출산",기쁨,감사하는,아내가 출산을 무사히 해줘서 기분이 좋아. 아기를 낳아준 아내에게 감사함을 느끼는 ...


In [ ]:
from google.colab import drive
import pandas as pd
import re
import os

# ======================================
# 0. 드라이브 마운트 & 데이터 경로
# ======================================
drive.mount('/content/drive')

DATA_DIR = "/content/drive/MyDrive/aiary/data"
train_path = os.path.join(DATA_DIR, "감성대화말뭉치(최종데이터)_Training.xlsx")
val_path   = os.path.join(DATA_DIR, "감성대화말뭉치(최종데이터)_Validation.xlsx")

train_df = pd.read_excel(train_path)
val_df   = pd.read_excel(val_path)

print("[LOG] train:", train_df.shape)
print("[LOG] val:", val_df.shape)


# ======================================
# 1. 대화 병합 함수
# ======================================
def merge_dialogue(row):
    fields = ["사람문장1","시스템문장1","사람문장2","시스템문장2","사람문장3","시스템문장3"]
    return " ".join(str(row[f]) for f in fields if f in row and pd.notna(row[f]))

train_df["dialogue"] = train_df.apply(merge_dialogue, axis=1)
val_df["dialogue"]   = val_df.apply(merge_dialogue, axis=1)


# ======================================
# 2. 어간 기반 키워드 정의
# ======================================
growth_action_stems = [
    "처음으로","첫","첫걸음","첫마디","첫발","성공","시작","도전",
    "발달","성장","뒤집기","배밀이","기기","기어가기",
    "걸음마","혼자서 걷","혼자 걷","옹알이","엄마라고","아빠라고",
    "이 나","이 났","이앓이","키가","키 재","몸무게","체중","측정",

    "먹","마시","한입","삼키","입에 넣",
    "자고","잠들","깨","하품","기지개",
    "울","울음","보채","투정","화내",
    "웃","웃음","방긋","활짝","표정","미소",

    "놀","장난치","뛰","걷","달리","기어",
    "넘어","일어나","앉","서","매달리",
    "던지","잡","쥐","만지","쌓","부수",
    "그리","그림","색칠","만들","조립",

    "입히","벗기","쓰","신기","들어올리","안기","업기","손잡",

    "놀이터","미끄럼틀","그네","모래놀이",
    "키즈카페","키카","트램펄린","볼풀장",
    "문센","문화센터","수업시간","교실",
    "목욕","샤워","물놀이","욕조","튜브",
    "유모차","카시트","보행기","점퍼루","쏘서","식탁의자",
    "식탁","밥상","의자","소파","쇼파",
    "침대","이불","베개","바닥","매트",
    "공원","잔디","산책","야외","실내",

    "블록","레고","퍼즐","책","초점책","딸랑이",
    "공갈젖꼭지","젖병","컵","스푼","포크","식판",
    "곰돌이","인형","자동차 장난감",

    "신나","즐거워","기뻐","무서워","놀라","좋아해",
    "호기심","궁금해","집중하","열중하","흥미로워"
]

abstract_exclude_stems = [
    "남편","신랑","아내","시댁","친정",
    "부모님", "부부", "와이프", "그녀",
    "면허", "시집", "처가", "친가", "갱년기",
    "친구","동기","동료","상사","직원","부장",
    "연애","소개팅","데이트","결혼","이혼",
    "아빠가 된다니", "죽은", "죽음", "아빠가 되면",
    "돌아가시", "돌아가셨", "돌아가셔", "돌아가신",

    "직장","회사","업무","보고서","취업","퇴사",
    "시험","과제","학업","대학교","학원", "성적",

    "돈","대출","카드값","주식","부동산","보험", "경제", "루머",

    "임신","출산 준비","출산 전","태교","뱃속",
    "결혼", "출산이 다가", "낳지", "낳을", "아이를 갖", "아이가 생겨",

    "초등학생","초등학교","중학생","중학교","고등학생","고등학교",
    "고1","고2","고3", "학교", "재수생", "반수",
    "대학생","대학원생", "대학", "명문대",

    # 입시/시험 관련
    "수능","입시","내신","모의고사","모고",
    "중간고사","기말고사","수시","정시",

    # 군대 관련
    "군대","입대","제대","훈련소","휴가 나왔","휴가 나와서","말년",

    # 알바/사회생활
    "알바","아르바이트","편의점 알바","카페 알바",

    # 사춘기/반항
    "사춘기","반항기"
]

child_target_words = [
    "아기","애기","아가","아이","우리 아이","우리애","우리 애",
    "아들","딸","아들래미","딸래미","막둥이","꼬맹이","조카"
]

# 상황키워드 중 육아 관련 값만 허용
parenting_situations = [
    "대인관계(부부, 자녀)",
    "가족관계",
    "연애,결혼,출산"
]


# ======================================
# 3. 정규식 변환 (어간 매칭)
# ======================================
def make_stem_pattern(stems):
    escaped = [re.escape(s) + r"\w*" for s in stems]
    return re.compile("(" + "|".join(escaped) + ")")

growth_pattern   = make_stem_pattern(growth_action_stems)
abstract_pattern = make_stem_pattern(abstract_exclude_stems)
child_pattern    = make_stem_pattern(child_target_words)


# ======================================
# 4. 필터링 함수
# ======================================
def is_child_growth_dialogue(text: str) -> bool:
    if not isinstance(text, str):
        return False

    # 성인/직장/연애/임신 등 추상/비육아 맥락이 있으면 제외
    if abstract_pattern.search(text):
        return False

    # 아이/자녀 지칭 단어 최소 1개
    if not child_pattern.search(text):
        return False

    # 성장/행동/상황 어간 최소 1개
    if not growth_pattern.search(text):
        return False

    return True

def is_parenting_situation(s):
    return isinstance(s, str) and s in parenting_situations


# ======================================
# 5. Training + Validation 필터링
# ======================================
train_mask_dialogue  = train_df["dialogue"].apply(is_child_growth_dialogue)
train_mask_situation = train_df["상황키워드"].apply(is_parenting_situation)
train_mask = train_mask_dialogue & train_mask_situation

val_mask_dialogue  = val_df["dialogue"].apply(is_child_growth_dialogue)
val_mask_situation = val_df["상황키워드"].apply(is_parenting_situation)
val_mask = val_mask_dialogue & val_mask_situation

train_child_growth = train_df[train_mask]
val_child_growth   = val_df[val_mask]

print("\n========== FILTER RESULT ==========")
print("[TRAIN] 전체:", len(train_df), " | 육아 성장 대화:", len(train_child_growth),
      " | 비율: {:.3f}%".format(len(train_child_growth)/len(train_df)*100))
print("[VAL]   전체:", len(val_df), " | 육아 성장 대화:", len(val_child_growth),
      " | 비율: {:.3f}%".format(len(val_child_growth)/len(val_df)*100))


# ======================================
# 6. 샘플 출력 (Training/Validation에서 3개씩)
# ======================================
def print_samples(df, name):
    print(f"\n===== {name} SAMPLE 3개 =====")
    for _, row in df.head(3).iterrows():
        print("─────────────────────────")
        print("상황:", row["상황키워드"])
        print("감정:", row["감정_대분류"], "/", row["감정_소분류"])
        print("대화:", row["dialogue"])
        print()

print_samples(train_child_growth, "TRAIN")
print_samples(val_child_growth, "VALIDATION")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
[LOG] train: (51630, 13)
[LOG] val: (6641, 13)

========== FILTER RESULT ==========
[TRAIN] 전체: 51630  | 육아 성장 대화: 629  | 비율: 1.218%
[VAL]   전체: 6641  | 육아 성장 대화: 90  | 비율: 1.355%

===== TRAIN SAMPLE 3개 =====
─────────────────────────
상황: 연애,결혼,출산
감정: 불안 / 조심스러운
대화: 아기를 겨우 재워서 조심스레 침대에 눕혔는데 또 깰까 봐 불안해. 힘들게 재운 아기가 깰까 봐 불안하시군요. 우리 아기가 작은 소리에도 깨는 민감한 아이라고 하더라고. 어떻게 하면 민감한 아기가 깨지 않고 잘 자는 데 도움을 줄 수 있을까요? 벨소리는 무조건 무음으로 바꿔 놓고 낮에는 많이 놀아 줘서 밤에 잘 잘 수 있게 도와줘야겠지. 아기가 깨지 않고 잘 자길 바라요.

─────────────────────────
상황: 대인관계(부부, 자녀)
감정: 기쁨 / 감사하는
대화: 우리 부자간은 사이가 좋아서 정말 감사해. 아드님과 사이가 좋아 감사함을 느끼시는군요. 날 닮아 무뚝뚝한 거 같지만 속으로 날 많이 챙기지. 무뚝뚝한 것 같지만 아드님이 많이 챙겨주는군요. 아드님과 사이가 좋은 비결이 있나요? 아들과 대화를 많이 하고 아들을 믿고 지지해줬어. 대화를 많이 나누고 지지해 주는 것이 비결이군요.

─────────────────────────
상황: 대인관계(부부, 자녀)
감정: 기쁨 / 자신하는
대화: 우리 아이들은 부모에게는 거짓말을 절대 하지 않아. 자녀들에 대한 신뢰가 크시군요. 내가 모르는 비밀이 있을 수도 있지만 우리 아이들은 언제나 솔직하게 이야기

In [ ]:
import json

OUTPUT_DIR = "/content/drive/MyDrive/aiary/data/감성대화말뭉치_filtered"
os.makedirs(OUTPUT_DIR, exist_ok=True)

train_json_path = os.path.join(OUTPUT_DIR, "filtered_train.json")
val_json_path   = os.path.join(OUTPUT_DIR, "filtered_val.json")

def save_json_list(df, path, prefix):
    items = []
    for i, row in df.iterrows():
        items.append({
            "id": f"{prefix}_{i}",
            "situation": row["상황키워드"],
            "emotion_big": row["감정_대분류"],
            "emotion_small": row["감정_소분류"],
            "dialogue": row["dialogue"]
        })

    with open(path, "w", encoding="utf-8") as f:
        json.dump(items, f, ensure_ascii=False, indent=2)

save_json_list(train_child_growth, train_json_path, "train")
save_json_list(val_child_growth, val_json_path, "val")

print("[SAVE] JSON 저장 완료")
print(" -", train_json_path)
print(" -", val_json_path)


NameError: name 'train_child_growth' is not defined

In [ ]:
def emotion_ratio(df, name):
    print(f"\n===== Emotion Ratio ({name}) =====")

    # 대분류 비율
    big = df["감정_대분류"].value_counts(normalize=True) * 100
    print("\n[감정 대분류 비율]")
    print(big.round(2))

    # 소분류 비율
    small = df["감정_소분류"].value_counts(normalize=True) * 100
    print("\n[감정 소분류 비율]")
    print(small.round(2))

# 실행
emotion_ratio(train_child_growth, "TRAIN")
emotion_ratio(val_child_growth, "VAL")



===== Emotion Ratio (TRAIN) =====

[감정 대분류 비율]
감정_대분류
슬픔    19.08
불안    18.76
기쁨    17.33
상처    16.53
분노    14.47
당황    13.83
Name: proportion, dtype: float64

[감정 소분류 비율]
감정_소분류
고립된            3.66
충격 받은          3.34
기쁨             3.34
혼란스러운          3.34
두려운            3.02
흥분             2.86
죄책감의           2.70
걱정스러운          2.70
비통한            2.54
슬픔             2.54
안도             2.54
우울한            2.54
조심스러운          2.23
성가신            2.23
억울한            2.23
짜증내는           2.07
실망한            2.07
한심한            2.07
후회되는           2.07
당혹스러운          1.91
환멸을 느끼는        1.91
부끄러운           1.91
분노             1.91
악의적인           1.91
눈물이 나는         1.91
만족스러운          1.91
편안한            1.75
상처             1.75
배신당한           1.59
감사하는           1.59
툴툴대는           1.59
희생된            1.43
질투하는           1.43
불안             1.43
스트레스 받는        1.43
초조한            1.43
회의적인           1.27
당황             1.27
안달하는           1.27
낙담한            1.27
자신하는           1.11


In [ ]:
# 상황키워드 유니크 값 출력
unique_situations = train_df["상황키워드"].dropna().unique()

print("상황키워드 개수:", len(unique_situations))
for v in unique_situations:
    print("-", v)


상황키워드 개수: 12
- 진로,취업,직장
- 대인관계
- 연애,결혼,출산
- 재정,은퇴,노후준비
- 재정
- 건강,죽음
- 학업 및 진로
- 학교폭력/따돌림
- 직장, 업무 스트레스
- 대인관계(부부, 자녀)
- 가족관계
- 건강


In [ ]:
from google.colab import userdata
real = userdata.get('real')

In [ ]:
import os

BASE_DIR = "/content/drive/MyDrive/aiary/data"

for root, dirs, files in os.walk(BASE_DIR):
    print("DIR:", root)
    print("   SUBDIRS:", dirs)
    print("   FILES:", files)
    print("------------------------------------------------")


DIR: /content/drive/MyDrive/aiary/data
   SUBDIRS: ['감성대화말뭉치_filtered', 'parenting_diary_gpt', 'parenting_diary_gpt_fail']
   FILES: ['감성대화말뭉치(최종데이터)_Validation.xlsx', 'diary_corpus.csv', 'diary_corpus_parenting_files_only.csv', 'diary_corpus_parenting_final.csv', 'diary_corpus_parenting_final_splitted.csv', 'diary_corpus_parenting_file_level.csv', 'diary_corpus_parenting_sentence_level.csv', '감성대화말뭉치(최종데이터)_Training.xlsx']
------------------------------------------------
DIR: /content/drive/MyDrive/aiary/data/감성대화말뭉치_filtered
   SUBDIRS: []
   FILES: ['filtered_train.json', 'filtered_val.json']
------------------------------------------------
DIR: /content/drive/MyDrive/aiary/data/parenting_diary_gpt
   SUBDIRS: []
   FILES: ['gpt_parenting_diary_train.csv', 'gpt_parenting_diary_val.csv', 'gpt_parenting_diary_val.jsonl', 'gpt_parenting_diary_train.jsonl']
------------------------------------------------
DIR: /content/drive/MyDriv

In [ ]:
from openai import OpenAI
import pandas as pd
import json
from tqdm.auto import tqdm
import time
import os

# ======================================
# 0. OpenAI 클라이언트 설정
# ======================================
client = OpenAI(api_key=real)
# client = OpenAI()

# ======================================
# 1. 필터링된 데이터 로드
#    (위에서 만든 filtered_train.json, filtered_val.json 사용)
# ======================================
BASE_DIR = "/content/drive/MyDrive/aiary/data/감성대화말뭉치_filtered"
train_json_path = os.path.join(BASE_DIR, "filtered_train.json")
val_json_path   = os.path.join(BASE_DIR, "filtered_val.json")

train_filtered = pd.read_json(train_json_path)
val_filtered   = pd.read_json(val_json_path)

print("[LOG] filtered train:", train_filtered.shape)
print("[LOG] filtered val:", val_filtered.shape)


# ======================================
# 2. GPT 프롬프트 / 호출 함수 정의
# ======================================

SYSTEM_PROMPT = """
너는 '육아 일기 데이터셋 정제기 및 각색 작가'다.
역할은 다음과 같다.

1) 입력으로 주어진 한국어 대화를 읽고, 이 대화가 0~7세(영유아/미취학 아동)를 대상으로 하는지 판별한다.
2) 0~7세 대상이라고 판단되는 경우에만, 30대 엄마가 쓰는 반말 육아 일기체로 자연스럽게 재구성한다.
3) 일기 내용에서, 컴퓨터 비전(CV) 모델이 보고 이해할 수 있을 만한 '화면에 보이는 사실'을 3~5줄로 건조하게 요약한다.
4) 새로운 사실(장소, 인물, 사건)을 만들어내지 말고, 입력 대화에서 알 수 있는 정보만 사용한다.
5) 나이가 애매하면 보수적으로 is_valid=false로 처리한다.
"""

def build_user_prompt(dialogue_text: str) -> str:
    return f"""
아래는 한 사람(부모)과 시스템이 주고받은 대화이다.

[Input Dialogue]
{dialogue_text}

[Step 1: 필터링]
이 대화의 대상(자녀)이 '0세~6세(영유아 및 미취학 아동)'라고 판단되는가?
- PASS 예시: 기저귀, 수유, 어린이집, 유치원 적응, 장난감, 떼쓰기, 낮잠/잠투정, 목욕, 놀이터, 키즈카페, 편식, 이유식, 유모차, 카시트, 낮잠/밤잠 등.
- FAIL 예시: 해외여행(혼자 다님), 커피 심부름, 학교 성적, 시험/수능/입시, 사춘기, 취업, 연애, 결혼, 군대, 알바, 대학 생활, 재수/반수 등.

[Step 2: 일기 변환 (PASS인 경우만)]
대화 내용을 바탕으로 '30대 엄마가 쓴 감성적인 육아 일기(반말 모놀로그)'를 작성해라.
- 1인칭 시점, 반말 일기체: "~했어", "~하네", "~했다" 등
- 문장은 자연스럽게 연결하되, 입력 대화에 없는 사실(장소, 인물, 사건 등)은 새로 만들지 마라.
- 아이의 행동, 표정, 몸짓, 주변 상황 등 시각적으로 떠오를 만한 묘사를 적절히 포함해라.

[Step 3: 한 줄 일기 생성 – PASS인 경우만]
Step 2에서 만든 일기 내용을 바탕으로, CV 모델이 인식했을 법한 화면 속 장면을 중심으로 하여, 그 모습을 은은한 감성으로 정리한 한 줄 일기 스타일 문장 3~5개를 생성하라.

각 문장은 단순한 시각적 요소 나열이 아니라,
보이는 모습을 바탕으로 한 짧고 부드러운 일기 표현이어야 한다.
감정은 과하게 표현하지 말고, 따뜻함·잔잔함 정도의 감성만 허용한다.

각 문장은 "1. ~", "2. ~" 형식으로 bullet 출력.

예시
"1. 아기가 포근한 이불 위에서 조용히 꿈나라에 빠져 있다."
"2. 작은 숨결이 리듬처럼 번져 방 안이 한층 더 따뜻해진다."
"3. 스탠드 불빛이 아이 얼굴을 은은하게 감싸고 있다."

[Output Format – JSON Only]

PASS인 경우(0~6세로 확실할 때):
{{
"is_valid": true,
"diary": "오늘 아기를 안고 한참을 달래다 겨우 잠들었어. 작게 켜둔 불 아래에서 아이 얼굴이 참 평온해 보이더라...",
"summary": "1. 아기가 포근한 이불 속에서 고르게 잠들어 있다.\n2. 작은 숨결이 방 안 공기를 따뜻하게 만든다.\n3. 은은한 조명이 아이 얼굴을 부드럽게 비춘다."
}}

FAIL인 경우(0~7세가 아니거나 애매할 때):
{{
  "is_valid": false,
  "reason": "성인 자녀(해외여행 언급)"
}}
"""


def process_with_gpt(row_text: str):
    user_prompt = build_user_prompt(row_text)
    try:
        resp = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": SYSTEM_PROMPT},
                {"role": "user", "content": user_prompt},
            ],
            response_format={"type": "json_object"},
            temperature=0.0,
        )
        content = resp.choices[0].message.content
        return json.loads(content)
    except Exception as e:
        # 에러 나면 로그만 남기고 fail 처리
        return {"is_valid": False, "error": str(e)}


# ======================================
# 3. GPT로 최종 육아 일기 데이터 생성
# ======================================

def build_final_dataset(df, split_name: str, max_rows=None):
    success_rows = []
    fail_rows = []

    it = df.iterrows()
    if max_rows is not None:
        it = df.head(max_rows).iterrows()

    print(f"[RUN] {split_name}: GPT 변환 시작 (rows={len(df) if max_rows is None else max_rows})")

    for idx, row in tqdm(it, total=(len(df) if max_rows is None else min(max_rows, len(df)))):
        dialogue = row["dialogue"]
        gpt_out = process_with_gpt(dialogue)

        if gpt_out.get("is_valid") is True:
            success_rows.append({
                "id": f"{split_name}_{idx}",
                "source_situation": row.get("situation", row.get("상황키워드", "")),
                "source_emotion_big": row.get("emotion_big", row.get("감정_대분류", "")),
                "source_emotion_small": row.get("emotion_small", row.get("감정_소분류", "")),
                "source_dialogue": dialogue,
                "diary": gpt_out.get("diary", ""),
                "summary": gpt_out.get("summary", ""),
            })
        else:
            # FAIL / ERROR 케이스 reason 모으기
            fail_rows.append({
                "id": f"{split_name}_{idx}",
                "source_situation": row.get("situation", row.get("상황키워드", "")),
                "source_emotion_big": row.get("emotion_big", row.get("감정_대분류", "")),
                "source_emotion_small": row.get("emotion_small", row.get("감정_소분류", "")),
                "source_dialogue": dialogue,
                "is_valid": gpt_out.get("is_valid", False),
                "reason": gpt_out.get("reason", ""),
                "error": gpt_out.get("error", "")
            })

    success_df = pd.DataFrame(success_rows)
    fail_df = pd.DataFrame(fail_rows)

    print(f"[DONE] {split_name}: is_valid=True 개수 = {len(success_df)}, is_valid=False/에러 = {len(fail_df)}")
    return success_df, fail_df



# 우선 train만 전체 돌리거나, 일부만 테스트해보고 싶으면 max_rows에 숫자 넣기
final_train_df, failed_train_df = build_final_dataset(train_filtered, "train")
final_val_df, failed_val_df     = build_final_dataset(val_filtered, "val")


# ======================================
# 4. 결과 저장 (CSV + JSONL)
# ======================================
OUT_DIR = "/content/drive/MyDrive/aiary/data/parenting_diary_gpt_v2"
os.makedirs(OUT_DIR, exist_ok=True)

train_csv_path   = os.path.join(OUT_DIR, "gpt_parenting_diary_train.csv")
val_csv_path     = os.path.join(OUT_DIR, "gpt_parenting_diary_val.csv")
train_jsonl_path = os.path.join(OUT_DIR, "gpt_parenting_diary_train.jsonl")
val_jsonl_path   = os.path.join(OUT_DIR, "gpt_parenting_diary_val.jsonl")

# CSV 저장
final_train_df.to_csv(train_csv_path, index=False)
final_val_df.to_csv(val_csv_path, index=False)

# JSONL 저장
def save_jsonl(df, path):
    with open(path, "w", encoding="utf-8") as f:
        for _, r in df.iterrows():
            f.write(json.dumps(dict(r), ensure_ascii=False) + "\n")

save_jsonl(final_train_df, train_jsonl_path)
save_jsonl(final_val_df, val_jsonl_path)

print("[SAVE] GPT 생성 육아 일기 저장 완료")
print(" -", train_csv_path)
print(" -", val_csv_path)
print(" -", train_jsonl_path)
print(" -", val_jsonl_path)

print("\n[HEAD - TRAIN]")
print(final_train_df.head())

FAIL_OUT_DIR = "/content/drive/MyDrive/aiary/data/parenting_diary_gpt_fail_v2"
os.makedirs(FAIL_OUT_DIR, exist_ok=True)

failed_train_csv = os.path.join(FAIL_OUT_DIR, "gpt_parenting_diary_train_fail.csv")
failed_val_csv   = os.path.join(FAIL_OUT_DIR, "gpt_parenting_diary_val_fail.csv")

failed_train_df.to_csv(failed_train_csv, index=False)
failed_val_df.to_csv(failed_val_csv, index=False)

print("[SAVE] FAIL 케이스 저장 완료")
print(" -", failed_train_csv)
print(" -", failed_val_csv)

# 샘플로 reason 몇 개 보기
print("\n[FAIL SAMPLE - TRAIN]")
print(failed_train_df[["source_dialogue", "reason", "error"]].head())



[LOG] filtered train: (629, 5)
[LOG] filtered val: (90, 5)
[RUN] train: GPT 변환 시작 (rows=629)


  0%|          | 0/629 [00:00<?, ?it/s]

[DONE] train: is_valid=True 개수 = 177, is_valid=False/에러 = 452
[RUN] val: GPT 변환 시작 (rows=90)


  0%|          | 0/90 [00:00<?, ?it/s]

[DONE] val: is_valid=True 개수 = 35, is_valid=False/에러 = 55
[SAVE] GPT 생성 육아 일기 저장 완료
 - /content/drive/MyDrive/aiary/data/parenting_diary_gpt_v2/gpt_parenting_diary_train.csv
 - /content/drive/MyDrive/aiary/data/parenting_diary_gpt_v2/gpt_parenting_diary_val.csv
 - /content/drive/MyDrive/aiary/data/parenting_diary_gpt_v2/gpt_parenting_diary_train.jsonl
 - /content/drive/MyDrive/aiary/data/parenting_diary_gpt_v2/gpt_parenting_diary_val.jsonl

[HEAD - TRAIN]
         id source_situation source_emotion_big source_emotion_small  \
0   train_0         연애,결혼,출산                 불안                조심스러운   
1   train_1     대인관계(부부, 자녀)                 기쁨                 감사하는   
2  train_10     대인관계(부부, 자녀)                 기쁨                만족스러운   
3  train_12     대인관계(부부, 자녀)                 당황                  외로운   
4  train_16         연애,결혼,출산                 상처                충격 받은   

                                     source_dialogue  \
0  아기를 겨우 재워서 조심스레 침대에 눕혔는데 또 깰까 봐 불안해. 힘들게 재운 아기..

In [ ]:
import pandas as pd
final_train_df = pd.read_csv("/content/drive/MyDrive/aiary/data/parenting_diary_gpt_v2/gpt_parenting_diary_train.csv")
final_val_df = pd.read_csv("/content/drive/MyDrive/aiary/data/parenting_diary_gpt_v2/gpt_parenting_diary_val.csv")


final_all = pd.concat([final_train_df, final_val_df])

OUT_DIR = "/content/drive/MyDrive/aiary/data/parenting_diary_gpt_v2"
final_all.to_csv("final_all.csv", index=False)

In [ ]:
final_all.shape

(212, 7)

In [ ]:
import os
import json
import random
import time
from datetime import datetime
from openai import OpenAI

# ======================================
# 0. 설정
# ======================================
API_KEY = real  # 예: "sk-..."
client = OpenAI(api_key=API_KEY)

MODEL_NAME = "gpt-4o-mini"  # 가성비 모델

# 저장 경로
OUTPUT_DIR = "/content/drive/MyDrive/aiary/data"  # Colab 기준 경로
os.makedirs(OUTPUT_DIR, exist_ok=True)

# 100개 토픽 리스트
TOPICS = [
    "이유식 먹는 아기", "이유식을 거부하는 아기", "스스로 과자를 집어 먹는 아기",
    "빨대컵으로 물을 마시는 아기", "젖병을 물고 누워 있는 아기", "아기띠에 안겨 산책하는 아기",
    "유모차에 앉아 밖을 구경하는 아기", "카시트에서 졸음 오는 아기", "바닥에서 배밀이하는 아기",
    "네 발로 기어 다니는 아기", "잡고 서기 연습하는 아기", "첫걸음을 떼는 아기",
    "소파를 붙잡고 서 있는 아기", "걸음마 보조기를 밀며 걷는 아기", "바닥에서 블록을 쌓는 아기",
    "레고 조각을 맞추는 아기", "장난감 자동차를 밀며 노는 아기", "기차 레일 장난감을 조립하는 아기",
    "원목 교구 장난감으로 손놀이하는 아기", "공을 굴리거나 던지며 노는 아기", "바닥에 앉아 그림책을 보는 아기",
    "책장에서 책을 꺼내는 아기", "스티커 붙이기 놀이하는 아기", "색연필로 낙서하는 아기",
    "식탁에서 물감놀이하는 아기", "클레이로 모양을 만드는 아기", "비눗방울을 잡으려 뛰는 아기",
    "인형을 끌어안고 있는 아기", "인형에게 젖병을 먹이는 아기", "곰돌이 인형과 낮잠 자는 아기",
    "침대에서 낮잠 자는 아기", "이불을 걷어차고 자는 아기", "담요 속에 파묻혀 있는 아기",
    "엄마 품에서 잠드는 아기", "잠 결린 얼굴로 일어나는 아기", "목욕 후 수건을 뒤집어쓴 아기",
    "욕조에서 물장구치는 아기", "머리를 감겨주는 아기", "세면대에서 양치하는 아기",
    "거울을 보며 표정을 따라 하는 아기", "기저귀를 갈고 있는 아기", "배변 성공 후 기뻐하는 아기",
    "손 씻기 연습하는 아기", "옷을 입히는 장면의 아기", "스스로 양말을 신어보는 아기",
    "신발을 신으려 하는 아기", "유치원 등원 준비 중인 아기", "유치원 가방을 메고 서 있는 아기",
    "등원길에 손잡고 걷는 아기", "하원 후 가방을 여는 아기", "유치원 작품을 들고 있는 아기",
    "생일 케이크를 바라보는 아기", "생일 촛불을 끄는 아기", "선물 상자를 뜯어보는 아기",
    "가족과 함께 사진 찍는 아기", "부모와 바닥에서 놀이하는 아기", "할머니와 함께 노는 아기",
    "형제자매와 장난감을 두고 다투는 아기", "형제자매와 웃으며 노는 아기", "강아지를 쓰다듬는 아기",
    "고양이를 따라다니는 아기", "반려동물에게 간식을 주는 아기", "키즈카페 볼풀장에서 노는 아기",
    "키즈카페 미끄럼틀 타는 아기", "키즈카페 의자에 앉아 쉬는 아기", "놀이터 그네 타는 아기",
    "놀이터 미끄럼틀 내려오는 아기", "모래 놀이터에서 모래를 퍼는 아기", "잔디밭을 뛰어다니는 아기",
    "공원 벤치에 앉아 쉬는 아기", "분수 앞에서 물을 구경하는 아기", "비 오는 날 우산을 쓰고 서 있는 아기",
    "장화 신고 물웅덩이를 밟는 아기", "눈사람 옆에 서 있는 아기", "눈덩이를 굴리는 아기",
    "크리스마스 트리 앞에서 노는 아기", "택배 상자를 열어보는 아기", "주방 냄비를 두드리며 노는 아기",
    "냉장고 문을 열어보는 아기", "소파에 앉아 TV를 보는 아기", "침대 위에서 방방 뛰는 아기",
    "이불 속에 숨어 얼굴을 내미는 아기", "창문 밖을 바라보는 아기", "커튼을 잡아당기며 노는 아기",
    "방바닥에 장난감을 늘어놓고 노는 아기", "찬장 아래를 탐색하는 아기", "아기 매트 위에서 장난감을 모으는 아기",
    "높게 쌓은 블록을 바라보는 아기", "소파 아래 장난감을 찾는 아기", "휴대폰이나 리모컨을 만지는 아기",
    "식탁에서 과일을 먹는 아기", "간식을 들고 뛰어오는 아기", "장난감을 바구니에 담아 옮기는 아기",
    "계단 난간을 잡고 올라가려는 아기", "장난감 책상에서 앉아 노는 아기", "마트 쇼핑카트에 앉아 있는 아기",
    "집 앞 마당에서 물놀이하는 아기", "풍선을 가지고 뛰어노는 아기", "화분 흙을 만져보는 아기",
    "작은 공을 굴려 따라가는 아기"
]

# 엄마의 컨디션 (일기의 톤앤매너 결정)
MOM_CONDITIONS = [
    # 1. [밝음/쾌활] 아이와 노는 게 즐겁고 에너지가 넘치는 톤
    "활기차고 기분 좋은 상태 (긍정적이고 밝은 말투)",

    # 2. [감성/차분] 아이가 자거나 조용할 때 느끼는 뭉클함
    "차분하고 감성적인 상태 (아이에 대한 애틋함)",

    # 3. [현실/극복] 육아는 힘들지만 아이의 미소로 치유되는 톤
    "몸은 지치고 힘들지만 아이를 보며 다시 힘을 내는 상태 (현실 육아)",

    # 4. [인내/사랑] 사고를 치거나 떼를 써도 사랑으로 감싸는 톤
    "육아의 어려움 속에서도 인내심과 사랑을 느끼는 상태",

    # 5. [평온/감사] 특별한 일은 없지만 소소한 일상에 감사하는 톤
    "평화롭고 여유로운 일상 속에서 행복을 느끼는 상태"
]

# ======================================
# GPT 호출 함수
# ======================================
def call_gpt(system_prompt, user_prompt, temperature=0.7):
    try:
        res = client.chat.completions.create(
            model=MODEL_NAME,
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user",    "content": user_prompt},
            ],
            temperature=temperature,
        )
        return res.choices[0].message.content.strip()
    except Exception as e:
        print(f"API Error: {e}")
        time.sleep(2)
        return None

# ======================================
# 메인 실행
# ======================================
def main():
    ts = datetime.now().strftime("%Y%m%d_%H%M%S")
    output_filename = os.path.join(OUTPUT_DIR, f"synthetic_parenting_data_dynamic_{ts}.jsonl")

    count = 0
    total_target = len(TOPICS) * 3

    # 샘플 출력용 카운터
    printed_samples = 0
    MAX_SAMPLE = 3

    print(f"[START] 총 {total_target}개 데이터 생성 시작...")
    print(f"[INFO] 저장 경로: {output_filename}")

    with open(output_filename, "w", encoding="utf-8") as f:
        for topic in TOPICS:
            for i in range(3):  # 토픽당 3세트

                # [핵심] 길이 모드(Length Mode) 랜덤 설정
                length_mode = random.choice(["SHORT", "MEDIUM", "LONG"])
                mom_condition = random.choice(MOM_CONDITIONS)

                # 모드별 설정 (요약 줄 수 & 일기 작성 지침)
                if length_mode == "SHORT":
                    target_summary_len = random.randint(2, 3)  # 요약 2~3줄
                    diary_instruction = "3~5문장 내외 (짧고 담백하게)"
                elif length_mode == "MEDIUM":
                    target_summary_len = random.randint(4, 5)  # 요약 4~5줄
                    diary_instruction = "6~8문장 내외 (보통 길이)"
                else:  # LONG
                    target_summary_len = random.randint(6, 8)  # 요약 6~8줄
                    diary_instruction = "9문장 이상 (풍성하고 디테일하게)"

                # -------------------------------
                # Step 2: 감성 일기 생성 (Target Data)
                # -------------------------------
                step2_system = (
                    "너는 0~6세 아이를 키우는 30대 엄마다. "
                    "주어진 상황, 컨디션, 그리고 길이 조건에 맞춰 "
                    "감성적인 반말 육아 일기를 써라. "
                    "사진에 보이는 시각적 묘사를 반드시 포함해야 한다."
                )

                step2_user = f"""
다음 장면을 기반으로 육아 일기를 작성하라.

- 사진 속 장면: {topic}
- 엄마의 상태: {mom_condition}
- **작성 길이: {diary_instruction}** (이 길이를 반드시 지킬 것)

[조건]
1. 말투: "~했어", "~네" (자연스러운 1인칭 독백)
2. 시각적 묘사 필수 (색깔, 위치, 동작 등)
3. 과장된 감정 표현 금지
"""
                diary = call_gpt(step2_system, step2_user, temperature=0.8)
                if not diary: continue

                # -------------------------------
                # Step 3: 감성 한 줄 일기 (Visual + Warm)
                # -------------------------------
                step3_system = (
                    "너는 육아 앨범에 들어갈 한 줄 코멘트를 작성한다. "
                    "가장 중요한 원칙은 '사진에 찍힌 것만 묘사한다'는 것이다. "
                    "엄마의 마음(영원함, 미래, 추억 등)이나 추상적인 분위기 묘사는 금지한다. "
                    "오직 아이의 행동, 표정, 옷차림을 따뜻한 형용사(귀여운, 앙증맞은, 해맑은)를 써서 표현하라."
                )

                step3_user = f"""
다음 긴 일기를 읽고, 그 안에 포함된 장면들을 정확히 {target_summary_len}개의 '한 줄 일기'로 변환하라.

[긴 일기]
{diary}

[작성 조건 - 매우 중요]
1. 형식: "1. ~", "2. ~" (총 {target_summary_len}줄)
2. 금지 사항: '영원히', '미래를 꿈꾸며', '에너지가 느껴진다', '고요함을 감싼다' 같은 추상적/문학적 표현 절대 금지.
3. 권장 사항: - "손가락으로 긁적이는 모습이 귀엽다" (O) - 행동+형용사
   - "입가에 묻은 노란 호박이 병아리 같네" (O) - 시각적 비유
   - "눈이 안 보이게 웃는 표정이 사랑스러워" (O) - 표정 묘사
"""
                summary = call_gpt(step3_system, step3_user, temperature=0.5)
                if not summary: continue

                # -------------------------------
                # 저장
                # -------------------------------
                obj = {
                    "id": f"syn_dyn_{count}",
                    "topic": topic,
                    "condition": mom_condition,
                    "length_mode": length_mode, # 나중에 분석용
                    "target_summary_len": target_summary_len,
                    "diary": diary,
                    "summary": summary,
                    "source": "synthetic"
                }

                f.write(json.dumps(obj, ensure_ascii=False) + "\n")

                # 샘플 출력
                if printed_samples < MAX_SAMPLE:
                    print(f"\n[Sample ({length_mode})]")
                    print(f"Topic: {topic}")
                    print(f"Summary ({target_summary_len}줄):\n{summary}")
                    print(f"Diary ({diary_instruction}):\n{diary[:100]}...")
                    printed_samples += 1

                count += 1
                if count % 10 == 0:
                    print(f"[PROGRESS] {count}/{total_target} 완료...")

    print(f"\n[DONE] 총 {count}개 생성 완료! 파일명: {output_filename}")

if __name__ == "__main__":
    main()

[START] 총 300개 데이터 생성 시작...
[INFO] 저장 경로: /content/drive/MyDrive/aiary/data/synthetic_parenting_data_dynamic_20251209_042738.jsonl

[Sample (SHORT)]
Topic: 이유식 먹는 아기
Summary (2줄):
1. 아기가 작은 입을 벌리며 호기심 가득한 눈빛으로 나를 바라보는 모습이 귀여워.  
2. 작은 손이 숟가락을 잡으려다 놓치고도 열심히 먹으려는 모습이 사랑스럽네.
Diary (3~5문장 내외 (짧고 담백하게)):
오늘은 주황색 당근 이유식을 처음 시도했어. 아기가 작은 입을 벌리며 호기심 가득한 눈빛으로 나를 바라보네. 고개를 살짝 숙이고 이유식을 기다리는 모습이 너무 귀여워서 미소가 절로...

[Sample (MEDIUM)]
Topic: 이유식 먹는 아기
Summary (5줄):
1. 아기가 주황색 이유식을 한 숟가락씩 먹으며 작은 손으로 스푼을 잡고 있는 모습이 귀엽다.  
2. 아기의 눈이 반짝반짝 빛나며 맛있게 먹는 모습이 사랑스럽다.  
3. 고개를 까딱거리며 '더 줘!' 하는 모습이 앙증맞아 미소가 지어진다.  
4. 주방의 햇살 아래 아기가 웃으며 이유식을 먹는 모습이 따뜻하게 느껴진다.  
5. 아기의 웃음소리가 울려 퍼지는 순간이 정말 소중하게 느껴진다.
Diary (6~8문장 내외 (보통 길이)):
오늘은 아기가 주황색 이유식을 한 숟가락씩 먹고 있었어. 그 작은 손으로 스푼을 잡고는 입에 가져가더니, 맛있는 듯 눈이 반짝반짝 빛났네. 나도 그 모습을 보면서 힘이 나더라. 몸...

[Sample (MEDIUM)]
Topic: 이유식 먹는 아기
Summary (5줄):
1. 아기가 작은 손으로 노란 이유식 그릇을 꼭 잡고 있는 모습이 귀엽다.  
2. 이유식을 입에 넣으려다가 얼굴에 묻힌 모습이 웃음 나게 한다.  
3. 아기의 입가에 묻은 하얗고 말랑말랑한 이유식이 사랑스럽다.  
4. 아기의 반짝이는 눈이 해맑고 기분